In [9]:
import pandas as pd
import os
import re

In [10]:
queries = pd.read_json("C:/Users/GODZILLA/Desktop/Faculdade/TCC/spider2/dev.json")
tables = pd.read_json("C:/Users/GODZILLA/Desktop/Faculdade/TCC/spider2/tables.json")
unique_tables = list(tables['db_id'].unique())
path = './spider2/database/'

In [11]:
def get_schema(row):
    schema = "Schema: "
    tmp = []
    tables = list(row['table_names_original'])
    column_names = list(row['column_names_original'])
    column_types = list(row['column_types'])
    foreign_keys = list(row['foreign_keys'])
    primary_keys = list(row['primary_keys'])
    column_nt = [[column_names[i], column_types[i].replace('number', 'int'), i, i in primary_keys, i in [x[0] for x in foreign_keys]] for i in range(len(column_names))]
    for i, t in enumerate(tables):
        schema = schema + '\'' + t + '\' ['
        columns = [name for name in column_nt if name[0][0] == i]
        for col in columns:
            if col[3] == True:
                schema = schema + ' Primary Key:'
            schema = schema + ' \'' + col[0][1] + '\' ' + col[1] + ','
        for fk in [x for x in columns if x[2] in [y[0] for y in foreign_keys]]:
            schema = schema + ' Foreign Key:' + ' \'' + fk[0][1] + '\' ' + fk[1] + ' from \'' + tables[column_names[fk[2]][0]] + '\' \'' + column_names[fk[2]][1] + '\','
        schema = schema[:-1] + '];\n'
    return schema

In [12]:
def get_input(row, tables):
    string = "Question: "
    string = string + row['question'] + '\n'
    string = string + tables[tables['db_id'] == row['db_id']]['schema'].values[0] + '\n'
    string = string + "Answer: " + row['query']
    return string

In [13]:
tables['schema'] = tables.apply(get_schema, axis = 1)

In [14]:
queries[0:1]

,db_id,query,query_toks,query_toks_no_value,question,question_toks,sql
0,concert_singer,SELECT count(*) FROM singer,"[SELECT, count, (, *, ), FROM, singer]","[select, count, (, *, ), from, singer]",How many singers do we have?,"[How, many, singers, do, we, have, ?]","{'from': {'table_units': [['table_unit', 1]], ..."


In [15]:

dataset = pd.DataFrame({"data":queries.apply(get_input, tables=tables, axis=1)})

In [16]:
dataset.to_json('dataset_dev.json', orient='records')